In [1]:
import torch
from torch import nn as nn
import os
import numpy as np
from tqdm.notebook import tqdm

In [2]:
def setup_torch(random_seed, use_gpu, gpu_number=0):
    torch.manual_seed(random_seed)
    torch.set_num_threads(8)
    if use_gpu:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_number)
        torch.cuda.manual_seed(random_seed)

In [3]:
setup_torch(0, 1, 0)

In [4]:
import json
import glob
import matplotlib.pyplot as plt

In [103]:
# loading our data
with open('/home/colin/features_cov/covid_feat_3bcnioqn_fold_0.json') as fp:
    all_data = json.load(fp)

In [104]:
train_x = []
val_x = []
test_x = []
train_y = []
val_y = []
test_y = []
all_x = []
for patient_id, data in all_data.items():
    if 'val_features' in data:
        val_x.append(data['val_features'])
        val_y.append(data['val_label'])
    elif 'test_features' in data:
        test_x.append(data['test_features'])
        test_y.append(data['test_label'])
    else:
        all_x += data['train_features']
        train_x.append(data['train_features'])
        train_y.append(data['train_label'])

In [155]:
class ClassifierMIL(nn.Module):
    
    def __init__(self):
        super(ClassifierMIL, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
#         self.classifier = nn.Linear(1024, 1)
        
    def forward(self, x):
        # x is BxNxF
        return torch.sigmoid(torch.mean(self.classifier(x)))

In [156]:
model = ClassifierMIL()
model.cuda()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fun = nn.BCELoss()
accum = 4

In [152]:
optim = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
all_accs = []
all_losses = []
for epoch in range(1000):
    accs = []
    losses =[]
    p_bar = tqdm(total=len(train_y))
    counter = 0
    ac_loss = 0
    inds = np.arange(0, len(train_x))
    np.random.shuffle(inds)
    for ind in inds:
        x_data = train_x[ind]
        y_data = train_y[ind]
        x_in = torch.tensor(x_data).view(1, -1, 1024).cuda()
        x_in = (x_in - 0.08)/ 0.021013880255015563
        y = torch.tensor(y_data).cuda()
        out = model(x_in)
        loss = loss_fun(out, y.float())
        ac_loss += loss
        acc = int(round(out.item()) == y_data)
        accs.append(acc)
        losses.append(float(loss))
        p_bar.set_description(f"loss = {np.mean(losses):3f}, acc = {np.mean(accs):.2f}")
        p_bar.update()
        counter += 1
        if counter % accum == 0:
            ac_loss /= counter
            optim.zero_grad()
            ac_loss.backward()
            optim.step()
            counter = 0
            ac_loss = 0
            
    if counter != 0:
        ac_loss /= counter
        optim.zero_grad()
        ac_loss.backward()
        optim.step()
        counter = 0
        ac_loss = 0
    print(f"Epoch {epoch}, Loss = {np.mean(losses)}, Acc = {np.mean(accs)}")
    all_accs.append(np.mean(accs))
    all_losses.append(np.mean(losses))

Epoch 0, Loss = 0.9887542265757642, Acc = 0.5833333333333334


Epoch 1, Loss = 0.637896426810095, Acc = 0.6538461538461539


Epoch 2, Loss = 0.5873449708884343, Acc = 0.6923076923076923


Epoch 3, Loss = 0.5733714470257744, Acc = 0.7051282051282052




Epoch 4, Loss = 0.5583826937318708, Acc = 0.7051282051282052


...

Epoch 95, Loss = 0.4789616402928144, Acc = 0.782051282051282
























Epoch 96, Loss = 0.47913604791168696, Acc = 0.7884615384615384


Epoch 97, Loss = 0.4807984949789571, Acc = 0.7884615384615384


Epoch 98, Loss = 0.48037882999088677, Acc = 0.7628205128205128


Epoch 99, Loss = 0.47920654249416356, Acc = 0.7756410256410257


In [132]:
out

tensor(0.0793, device='cuda:0', grad_fn=<SigmoidBackward>)

In [ ]:
plt.plot(all_accs)

In [ ]:
plt.plot(all_losses[100:])

In [141]:
inds = np.arange(0, len(val_x))
np.random.shuffle(inds)
accs = []
for ind in inds:
    x_data = val_x[ind]
    y_data = val_y[ind]
    x_in = torch.tensor(x_data).view(1, -1, 1024).cuda()
    x_in = (x_in - 0.08)/ 0.021013880255015563
    y = torch.tensor(y_data).cuda()
    out = model(x_in)
    loss = loss_fun(out, y.float())
    ac_loss += loss
    acc = int(round(out.item()) == y_data)
    accs.append(acc)

In [142]:
np.mean(accs)

0.7948717948717948

In [32]:
torch.tensor(train_x[0:8]).shape

ValueError: expected sequence of length 142 at dim 1 (got 129)

In [33]:
out = model(torch.tensor(train_x[0:1]))

In [35]:
out

tensor([[-0.4763]], grad_fn=<AddmmBackward>)

In [9]:
learned_vec = torch.zeros((1,1024))

In [16]:
r = learned_vec.view(1, 1, -1).repeat(3,1,1)

In [17]:
r.shape

torch.Size([3, 1, 1024])

In [20]:
torch.cat([r,r], axis=1).shape

torch.Size([3, 2, 1024])